In [1]:
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn import linear_model

In [2]:
memi171x_df = pd.read_csv("dataset/memi171x.csv", header=0)
memi172_df = pd.read_csv("dataset/memi172.csv", header=0)
memi173_df = pd.read_csv("dataset/memi173.csv", header=0)
memi174_df = pd.read_csv("dataset/memi174.csv", header=0)


In [3]:
fmli171x_df = pd.read_csv("dataset/fmli171x.csv", header=0)
fmli172_df = pd.read_csv("dataset/fmli172.csv", header=0)
fmli173_df = pd.read_csv("dataset/fmli173.csv", header=0)
fmli174_df = pd.read_csv("dataset/fmli174.csv", header=0)

In [4]:
df_171x = memi171x_df.groupby(['NEWID']).count()['AGE'].reset_index()
df_172 = memi172_df.groupby(['NEWID']).count()['AGE'].reset_index()
df_173 = memi173_df.groupby(['NEWID']).count()['AGE'].reset_index()
df_174 = memi174_df.groupby(['NEWID']).count()['AGE'].reset_index()

df_1_2 = df_171x.join(df_172, how='inner', on='NEWID', rsuffix='_other')
print("df_1_2:",df_1_2.shape)

df_2_3 = df_172.join(df_173, how='inner', on='NEWID', rsuffix='_other')
print("df_2_3:",df_2_3.shape)

df_3_4 = df_173.join(df_174, how='inner', on='NEWID', rsuffix='_other')
print("df_3_4:",df_3_4.shape)

df_1_2: (0, 4)
df_2_3: (0, 4)
df_3_4: (0, 4)


In [5]:
df_171x_mem = memi171x_df.groupby(['NEWID']).count()['AGE'].reset_index()
df_171x_fml = fmli171x_df.groupby(['NEWID']).count()['AGE2'].reset_index()

df_171x_mem_fml = df_171x_mem.join(df_171x_fml, how='inner', on='NEWID', rsuffix='_other')
print("df_171x_mem_fml:", df_171x_mem_fml.shape)

df_171x_mem_fml: (0, 4)


In [6]:
df_172_mem = memi172_df.groupby(['NEWID']).count()['AGE'].reset_index()
df_172_fml = fmli172_df.groupby(['NEWID']).count()['AGE2'].reset_index()

df_172_mem_fml = df_172_mem.join(df_172_fml, how='inner', on='NEWID', rsuffix='_other')
print("df_172_mem_fml:", df_172_mem_fml.shape)

df_172_mem_fml: (0, 4)


In [7]:
newid_list = set(memi171x_df['NEWID'].unique())
print(len(newid_list))

6208


In [8]:
mem_list = list(memi171x_df['NEWID'].unique())\
           +list(memi172_df['NEWID'].unique())\
           +list(memi173_df['NEWID'].unique())\
           +list(memi174_df['NEWID'].unique())
print(len(mem_list))        

24479


In [9]:
fml_list = list(fmli171x_df['NEWID'].unique())\
           +list(fmli172_df['NEWID'].unique())\
           +list(fmli173_df['NEWID'].unique())\
           +list(fmli174_df['NEWID'].unique())
print(len(fml_list))  

24479


In [10]:
join_list = set(mem_list) & set(fml_list)
print(len(join_list))

24479


In [11]:
#get the columns not in memi171x_df 
set(memi172_df.columns) - set(memi171x_df.columns) 

{'TRANAMTX', 'TRANDAYX', 'TRANPD', 'TRANPD_', 'TRAN_AYX', 'TRAN_MTX'}

In [12]:
#remove the 6 columns from all member dataframe
memi172_df = memi172_df.drop(columns=['TRANAMTX', 'TRANDAYX', 'TRANPD', 'TRANPD_', 'TRAN_AYX', 'TRAN_MTX'])
memi173_df = memi173_df.drop(columns=['TRANAMTX', 'TRANDAYX', 'TRANPD', 'TRANPD_', 'TRAN_AYX', 'TRAN_MTX'])
memi174_df = memi174_df.drop(columns=['TRANAMTX', 'TRANDAYX', 'TRANPD', 'TRANPD_', 'TRAN_AYX', 'TRAN_MTX'])
print(set(memi172_df.columns) - set(memi171x_df.columns))
print(set(memi173_df.columns) - set(memi171x_df.columns))
print(set(memi174_df.columns) - set(memi171x_df.columns))




set()
set()
set()


In [13]:
memi_full= pd.concat([memi171x_df, memi172_df, memi173_df, memi174_df])
print(memi_full.shape)

(59474, 181)


In [14]:
#all SALARYX_
memi_full[(memi_full['SALARYX_']=='D')].shape

(18885, 181)

In [15]:
#only "D" is valid 'SALARYX_". set invalid 'SALARYX' to 0, 
memi_full.loc[memi_full['SALARYX_'] != "D", "SALARYX"] = 0
#validate the assignment
print(memi_full[(memi_full['SALARYX_']!='D') & (memi_full['SALARYX'])>0][['SALARYX_', 'SALARYX']])

#only "D" is valid 'SEMP_RMX". set invalid 'SEMPFRMX' to 0, 
memi_full.loc[memi_full['SEMP_RMX'] != "D", "SEMPFRMX"] = 0
#validate the assignment
print(memi_full[(memi_full['SEMP_RMX']!='D') & (memi_full['SEMPFRMX'])>0][['SEMP_RMX', 'SEMPFRMX']])


#only "D" is valid 'SOCRRX_". set invalid 'SOCRRX' to 0, 
memi_full.loc[memi_full['SOCRRX_'] != "D", "SOCRRX"] = 0
#validate the assignment
print(memi_full[(memi_full['SOCRRX_']!='D') & (memi_full['SOCRRX'])>0][['SOCRRX_', 'SOCRRX']])

#only "D" is valid 'SSIX_". set invalid 'SSIX' to 0, 
memi_full.loc[memi_full['SSIX_'] != "D", "SSIX"] = 0
#validate the assignment
print(memi_full[(memi_full['SSIX_']!='D') & (memi_full['SSIX'])>0][['SSIX_', 'SSIX']])

Empty DataFrame
Columns: [SALARYX_, SALARYX]
Index: []
Empty DataFrame
Columns: [SEMP_RMX, SEMPFRMX]
Index: []
Empty DataFrame
Columns: [SOCRRX_, SOCRRX]
Index: []
Empty DataFrame
Columns: [SSIX_, SSIX]
Index: []


In [16]:
# each NEWID, aggregated 4 income sources.
memi_sum = memi_full.groupby("NEWID").agg({'SALARYX':'sum', 'SEMPFRMX':'sum',  'SOCRRX':'sum', 'SSIX':'sum'} )

In [17]:
print(fmli171x_df.shape)
print(fmli172_df.shape)
print(fmli173_df.shape)
print(fmli174_df.shape)

(6208, 808)
(6177, 828)
(6090, 828)
(6004, 828)


In [18]:
fml_cols_remove = list(set(fmli172_df.columns) - set(fmli171x_df.columns))
print(fml_cols_remove)

['IRAYR_', 'IRA', 'STDNTYR', 'WHLFYR', 'IRAYR', 'FFTAXOWE', 'CREDYR_', 'STOCKYR_', 'OTHLNYR', 'LIQUID_', 'STDNTYR_', 'SOLARPNL', 'FSTAXOWE', 'SOLA_PNL', 'WHLFYR_', 'STOCKYR', 'IRA_', 'CREDYR', 'LIQUDYR_', 'OTHSTYR_', 'OTHSTYR', 'LIQUDYR', 'LIQUID', 'OTHLNYR_']


In [19]:
#remove the 6 columns from all member dataframe
fmli172_df = fmli172_df.drop(columns=fml_cols_remove)
fmli173_df = fmli173_df.drop(columns=fml_cols_remove)
fmli174_df = fmli174_df.drop(columns=fml_cols_remove)
print(set(fmli172_df.columns) - set(fmli171x_df.columns))
print(set(fmli173_df.columns) - set(fmli171x_df.columns))
print(set(fmli174_df.columns) - set(fmli171x_df.columns))

set()
set()
set()


In [20]:
fmli_full= pd.concat([fmli171x_df, fmli172_df, fmli173_df, fmli174_df])
print(fmli_full.shape)

(24479, 808)


In [21]:
def processMortgage(row):
    total = 0
    
    if row.MRTINTCQ >0:
        total +=row.MRTINTCQ
    else:
        total +=row.MRTINTPQ
        
    if row.RNTXRPCQ >0:
        total +=row.RNTXRPCQ
    else:
        total +=row.RNTXRPPQ
        
    if row.PROPTXCQ >0:
        total +=row.PROPTXCQ
    else:
        total +=row.PROPTXPQ
   
    return total*4

fmli_full['Mortgage_Rent_Property_Tax'] = fmli_full.apply(processMortgage, axis=1)

In [22]:
fmli_mortgage = fmli_full[['NEWID', 'MRTINTCQ', 'MRTINTPQ', 'RNTXRPCQ', 'RNTXRPPQ', 'PROPTXCQ', 'PROPTXPQ', 'Mortgage_Rent_Property_Tax']]
fmli_mortgage.head(5)

,NEWID,MRTINTCQ,MRTINTPQ,RNTXRPCQ,RNTXRPPQ,PROPTXCQ,PROPTXPQ,Mortgage_Rent_Property_Tax
0,3386764,0.0,0.0,0.0,1510.0,0.0,0.0,6040.0
1,3386774,0.0,0.0,0.0,1689.0,0.0,0.0,6756.0
2,3386804,0.0,0.0,0.0,3300.0,0.0,0.0,13200.0
3,3386874,0.0,6717.0,0.0,0.0,0.0,350.0,28268.0
4,3386884,0.0,394.0,0.0,0.0,0.0,750.0,4576.0


In [23]:
def processHouseholdFurnishing(row):
    if row.HOUSEQCQ >0:
        return row.HOUSEQCQ*4
    else:
        return row.HOUSEQPQ*4
    
fmli_full['Household_Furnishings_Equipment'] = fmli_full.apply(processHouseholdFurnishing, axis=1)

def processHouseholdOps(row):
    if row.HOUSOPCQ >0:
        return row.HOUSOPCQ*4
    else:
        return row.HOUSOPPQ*4
    
fmli_full['Household_Operations'] = fmli_full.apply(processHouseholdOps, axis=1)


def processUtilities(row):
    if row.UTILCQ >0:
        return row.UTILCQ*4
    else:
        return row.UTILPQ*4
    
fmli_full['Utilities'] = fmli_full.apply(processUtilities, axis=1)


In [24]:
def processVehiclePurchases(row):
    total = 0
    
    if row.CARTKNCQ >0:
        total +=row.CARTKNCQ
    else:
        total +=row.CARTKNPQ
        
    if row.CARTKUCQ >0:
        total +=row.CARTKUCQ
    else:
        total +=row.CARTKUPQ
        
    if row.OTHVEHCQ >0:
        total +=row.OTHVEHCQ
    else:
        total +=row.OTHVEHPQ
   
    return total*4

fmli_full['Vehicle_Purchases'] = fmli_full.apply(processVehiclePurchases, axis=1)
        

In [25]:
 def processGas(row):
    if row.GASMOCQ >0:
        return row.GASMOCQ*4
    else:
        return row.GASMOPQ*4
    
fmli_full['Gasoline_Other_fuels_motor_oil'] = fmli_full.apply(processGas, axis=1)

def processVehicleExpenses(row):
    total = 0
    
    if row.VEHFINCQ >0:
        total +=row.VEHFINCQ
    else:
        total +=row.VEHFINPQ
        
    if row.MAINRPCQ >0:
        total +=row.MAINRPCQ
    else:
        total +=row.MAINRPPQ
        
    if row.VEHINSCQ >0:
        total +=row.VEHINSCQ
    else:
        total +=row.VEHINSPQ

    if row.VRNTLOCQ >0:
        total +=row.VRNTLOCQ
    else:
        total +=row.VRNTLOPQ
   
    return total*4

fmli_full['Other_vehicle_expenses'] = fmli_full.apply(processVehicleExpenses, axis=1)


In [26]:
def processPublicTransportation(row):
    if row.PUBTRACQ >0:
        return row.PUBTRACQ*4
    else:
        return row.PUBTRAPQ*4
    
fmli_full['Public_other_transportation'] = fmli_full.apply(processPublicTransportation, axis=1)

def processHealthInsurance(row):
    if row.HLTHINCQ >0:
        return row.HLTHINCQ*4
    else:
        return row.HLTHINPQ*4
    
fmli_full['Health_Insurance'] = fmli_full.apply(processHealthInsurance, axis=1)

def processMedicalServices(row):
    if row.MEDSRVCQ >0:
        return row.MEDSRVCQ*4
    else:
        return row.MEDSRVPQ*4
    
fmli_full['Medical_Services'] = fmli_full.apply(processMedicalServices, axis=1)

def processPersonalInsurance(row):
    if row.LIFINSCQ >0:
        return row.LIFINSCQ*4
    else:
        return row.LIFINSPQ*4
fmli_full['Personal_Insurance'] = fmli_full.apply(processPersonalInsurance, axis=1)

    
def processMedicalSupplies(row):
    if row.MEDSUPCQ >0:
        return row.MEDSUPCQ*4
    else:
        return row.MEDSUPPQ*4
fmli_full['Medical_Supplies'] = fmli_full.apply(processMedicalSupplies, axis=1)

def processPrescriptionDrugs(row):
    if row.PREDRGCQ >0:
        return row.PREDRGCQ*4
    else:
        return row.PREDRGPQ*4
fmli_full['Prescription_Drugs'] = fmli_full.apply(processPrescriptionDrugs, axis=1)
            
def processPensionSS(row):
    if row.RETPENCQ >0:
        return row.RETPENCQ*4
    else:
        return row.RETPENPQ*4
    
fmli_full['Pensions_and_Social_Security'] = fmli_full.apply(processPensionSS, axis=1)

def processFoodAtHome(row):
    if row.FDHOMECQ >0:
        return row.FDHOMECQ*4
    else:
        return row.FDHOMEPQ*4
    
fmli_full['Food_At_Home'] = fmli_full.apply(processFoodAtHome, axis=1)

def processFoodAwayFromHome(row):
    if row.FDAWAYCQ >0:
        return row.FDAWAYCQ*4
    else:
        return row.FDAWAYPQ*4
    
fmli_full['Food_Away_From_Home'] = fmli_full.apply(processFoodAwayFromHome, axis=1)

           
            

In [27]:
def processApparelServices(row):
    if row.APPARCQ >0:
        return row.APPARCQ*4
    else:
        return row.APPARPQ*4
    
fmli_full['Apparel_and_services'] = fmli_full.apply(processApparelServices, axis=1)        


def processPersonalCare(row):
    if row.PERSCACQ >0:
        return row.PERSCACQ*4
    else:
        return row.PERSCAPQ*4
    
fmli_full['Personal_Care'] = fmli_full.apply(processPersonalCare, axis=1)        
            
def processEducation(row):
    if row.EDUCACQ >0:
        return row.EDUCACQ*4
    else:
        return row.EDUCAPQ*4
    
fmli_full['Education'] = fmli_full.apply(processEducation, axis=1)            
            
def processCashContributions(row):
    if row.CASHCOCQ >0:
        return row.CASHCOCQ*4
    else:
        return row.CASHCOPQ*4
    
fmli_full['Cash_Contributions'] = fmli_full.apply(processCashContributions, axis=1)
            
            
            
def processMiscellaneous(row):
    if row.MISCCQ >0:
        return row.MISCCQ*4
    else:
        return row.MISCPQ*4
    
fmli_full['Miscellaneous'] = fmli_full.apply(processMiscellaneous, axis=1)
            
def processEntertainment(row):
    if row.ENTERTCQ >0:
        return row.ENTERTCQ*4
    else:
        return row.ENTERTPQ*4
    
fmli_full['Entertainment'] = fmli_full.apply(processEntertainment, axis=1)

            
def processTravel(row):
    if row.TTOTALC >0:
        return row.TTOTALC*4
    else:
        return row.TTOTALP*4
    
fmli_full['Travel'] = fmli_full.apply(processTravel, axis=1)
            

In [28]:
def column_fillna(colname):
    null_records = fmli_full[fmli_full[colname].isnull()].shape
    notnull_records = fmli_full[~fmli_full[colname].isnull()].shape
    fmli_full[colname] = fmli_full[colname].fillna(0)

    #verify fillna with 0,
    zero_records = fmli_full[fmli_full[colname]==0].shape
    nonzero_records = fmli_full[fmli_full[colname]!=0].shape
    #print(null_records)
    #print(notnull_records)
    #print(zero_records)
    #print(nonzero_records)
    
    
    if (null_records!=zero_records) or (notnull_records!=nonzero_records) :
        print("fillna failed")

In [29]:
column_fillna('INTRDVX')
column_fillna('INTRDVBX')

In [30]:
column_fillna('ROYESTX')
column_fillna('ROYESTBX')
column_fillna('OTHREGX')
column_fillna('OTHREGBX')
column_fillna('WELFAREX')
column_fillna('WELFREBX')
column_fillna('RETSURVX')
column_fillna('RETSRVBX')
column_fillna('NETRENTX')
column_fillna('NETRNTBX')
column_fillna('OTHRINCX')
     

fillna failed


In [31]:
col_HHTotalIncome=['INTRDVX', 'INTRDVBX', 'ROYESTX', 'ROYESTBX', 'OTHREGX', 'OTHREGBX',  'WELFAREX', 'WELFREBX', 
'RETSURVX', 'RETSRVBX', 'NETRENTX', 'NETRNTBX', 'OTHRINCX']

fmli_full['FMLI_HH_Total_Income'] = 0
for col in col_HHTotalIncome:
    fmli_full['FMLI_HH_Total_Income'] += fmli_full[col] 


In [32]:
 fmli_full['FMLI_HH_Total_Income'][:10]

0        0.0
1        1.0
2        0.0
3      450.0
4      228.0
5        0.0
6        0.0
7        0.0
8    13200.0
9    15600.0
Name: FMLI_HH_Total_Income, dtype: float64

In [33]:
fmli_cols=['NEWID', 'FMLI_HH_Total_Income',  'Mortgage_Rent_Property_Tax','Household_Furnishings_Equipment',
            'Household_Operations','Utilities','Vehicle_Purchases','Gasoline_Other_fuels_motor_oil','Other_vehicle_expenses',
            'Public_other_transportation','Health_Insurance','Medical_Services','Prescription_Drugs','Medical_Supplies',     
            'Personal_Insurance','Pensions_and_Social_Security','Food_At_Home','Food_Away_From_Home','Apparel_and_services',
            'Personal_Care','Education','Cash_Contributions','Miscellaneous','Entertainment','Travel']
fmli_cleaned = fmli_full[fmli_cols]

In [34]:
fmli_multi= fmli_full.groupby('NEWID').filter(lambda group: len(group) > 1)
memi_multi= memi_full.groupby('NEWID').filter(lambda group: len(group) > 1)

In [35]:
memi_sum['Total_Income'] = memi_sum['SALARYX'] + memi_sum['SEMPFRMX']+ memi_sum['SOCRRX']+ memi_sum['SSIX']
memi_sum = memi_sum[['Total_Income']] 
memi_sum.head(5)

,Total_Income
NEWID,
3386764,0.0
3386774,27916.0
3386804,65000.0
3386874,0.0
3386884,30768.0


In [36]:
fmli_memi = fmli_cleaned.join(memi_sum, how='inner', on='NEWID', rsuffix='_other')
print(fmli_memi.shape)

(24479, 26)


In [37]:
fmli_memi['HH_Total_Income'] = fmli_memi['Total_Income']+ fmli_memi['FMLI_HH_Total_Income']
fmli_memi.head(2)

,NEWID,FMLI_HH_Total_Income,Mortgage_Rent_Property_Tax,Household_Furnishings_Equipment,Household_Operations,Utilities,Vehicle_Purchases,Gasoline_Other_fuels_motor_oil,Other_vehicle_expenses,Public_other_transportation,...,Food_Away_From_Home,Apparel_and_services,Personal_Care,Education,Cash_Contributions,Miscellaneous,Entertainment,Travel,Total_Income,HH_Total_Income
0,3386764,0.0,6040.0,560.0,684.0,5324.0,0.0,1980.0,11600.0,0.0,...,200.0,680.0,240.0,15200.0,48772.0,0.0,900.0,1140.0,0.0,0.0
1,3386774,1.0,6756.0,60.0,732.0,1860.0,0.0,960.0,1824.0,0.0,...,520.0,368.0,200.0,0.0,1008.0,0.0,280.0,0.0,27916.0,27917.0


In [47]:
dependents=['Mortgage_Rent_Property_Tax','Household_Furnishings_Equipment',
            'Household_Operations','Utilities','Vehicle_Purchases','Gasoline_Other_fuels_motor_oil','Other_vehicle_expenses',
            'Public_other_transportation','Health_Insurance','Medical_Services','Prescription_Drugs','Medical_Supplies',     
            'Personal_Insurance','Pensions_and_Social_Security','Food_At_Home','Food_Away_From_Home','Apparel_and_services',
            'Personal_Care','Education','Cash_Contributions','Miscellaneous','Entertainment','Travel']


In [48]:
#dependents=['Mortgage_Rent_Property_Tax', 'Household_Furnishings_Equipment']
X=pd.DataFrame(fmli_memi['HH_Total_Income'])

results = []
for dependent in dependents:
    y=fmli_memi[dependent]
    model = linear_model.LinearRegression().fit(X, y)
    results.append([dependent, model.coef_[0], model.intercept_])

coefficients_df = pd.DataFrame(results, columns=['category', 'slope', 'intercept'])
print(coefficients_df)

coefficients_df.to_csv('dataset/coefficients.csv', index=False) 

                           category     slope    intercept
0        Mortgage_Rent_Property_Tax  0.025363  5025.920456
1   Household_Furnishings_Equipment  0.007068   922.350112
2              Household_Operations  0.007692   575.195225
3                         Utilities  0.007424  2206.189695
4                 Vehicle_Purchases  0.026389  2630.084953
5    Gasoline_Other_fuels_motor_oil  0.005730  1039.699158
6            Other_vehicle_expenses  0.013949  1871.551657
7       Public_other_transportation  0.006265   288.343316
8                  Health_Insurance  0.010704  1350.609107
9                  Medical_Services  0.005290   509.778498
10               Prescription_Drugs  0.000724   178.787455
11                 Medical_Supplies  0.000561    67.048843
12               Personal_Insurance  0.003950   202.388107
13     Pensions_and_Social_Security  0.054457  1792.073723
14                     Food_At_Home  0.009380  3168.457502
15              Food_Away_From_Home  0.010496  1332.1552

In [52]:
statedata_df = pd.read_csv("dataset/state_data.csv")
print(statedata_df.head(2))
coefficients_df= pd.read_csv("dataset/coefficients.csv")
print(coefficients_df.head(2))

  StateCode  DollarMultiplier  ExpenseRation
0        AL              1.15           0.79
1        AK              0.95           1.18
                          category     slope    intercept
0       Mortgage_Rent_Property_Tax  0.025363  5025.920456
1  Household_Furnishings_Equipment  0.007068   922.350112


In [108]:
def calculateExpenses(row, salary, dollar_multiplier, expense_ration):
    return (salary*dollar_multiplier*row['slope']+row['intercept'])*expense_ration/12
        
def predictRetirementExpesens(salary, currentState, retirementState):
    dollar_multiplier = statedata_df[statedata_df['StateCode']==currentState]['DollarMultiplier']
    expense_ration = statedata_df[statedata_df['StateCode']==retirementState]['ExpenseRation']
    coefficients_df['MonthlyExpense'] = coefficients_df.apply(calculateExpenses, args=(salary, dollar_multiplier.iloc[0],expense_ration.iloc[0]), axis=1)
    
    return coefficients_df['MonthlyExpense'] 
  
    

In [109]:
monthly_expense = predictRetirementExpesens(100000, "MI", "HI")
print(monthly_expense)

0     728.830240
1     158.073606
2     131.664765
3     282.554934
4     513.561481
5     155.635741
6     316.781163
7      90.279334
8     235.035425
9     101.303743
10     24.127075
11     11.962477
12     58.854931
13    717.448711
14    392.875363
15    231.200567
16     94.371576
17     31.218468
18    149.621523
19    199.591443
20     59.028048
21    244.706726
22    248.103261
Name: MonthlyExpense, dtype: float64
